In [46]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Конфиг

In [47]:
dataset = 'keypoint.csv'
model_save_path = 'keypoint_classifier.hdf5'
tflite_save_path = 'keypoint_classifier.tflite'

NUM_CLASSES = 4

In [48]:
X_dataset = pd.read_csv(dataset, delimiter=',', dtype='float32', usecols=list(range(1, 43)), header=None)
y_dataset = pd.read_csv(dataset, delimiter=',', dtype='int32', usecols=[0], header=None)

X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=RANDOM_SEED)

# Построение модели

In [49]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [50]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [51]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Обучение

In [52]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
22/30 [=====================>........] - ETA: 0s - loss: 1.5649 - accuracy: 0.1428
Epoch 1: saving model to keypoint_classifier.hdf5
30/30 [==============================] - 2s 14ms/step - loss: 1.5285 - accuracy: 0.1618 - val_loss: 1.3241 - val_accuracy: 0.3227
Epoch 2/1000
18/30 [=================>............] - ETA: 0s - loss: 1.3316 - accuracy: 0.2960
Epoch 2: saving model to keypoint_classifier.hdf5
30/30 [==============================] - 0s 5ms/step - loss: 1.3197 - accuracy: 0.3063 - val_loss: 1.2487 - val_accuracy: 0.3673
Epoch 3/1000
16/30 [===============>..............] - ETA: 0s - loss: 1.2628 - accuracy: 0.3516
Epoch 3: saving model to keypoint_classifier.hdf5
30/30 [==============================] - 0s 5ms/step - loss: 1.2614 - accuracy: 0.3504 - val_loss: 1.2117 - val_accuracy: 0.4108
Epoch 4/1000
18/30 [=================>............] - ETA: 0s - loss: 1.2346 - accuracy: 0.3676
Epoch 4: saving model to keypoint_classifier.hdf5
30/30 [=====================

In [53]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

8/8 [==============================] - 0s 3ms/step - loss: 0.2111 - accuracy: 0.9522


In [54]:
predict_result = model.predict(np.asarray(X_test.iloc[[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 55ms/step
[0.0881797  0.75452375 0.12698787 0.03030867]
1


# Classification report

In [55]:
from sklearn.metrics import classification_report

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print(classification_report(y_test, y_pred))

30/30 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       313
           1       0.98      0.91      0.95       305
           2       0.88      0.98      0.93       266
           3       0.98      0.83      0.90        58

    accuracy                           0.95       942
   macro avg       0.96      0.93      0.94       942
weighted avg       0.96      0.95      0.95       942



# Конвертация в tensorflow-lite

In [56]:
model.save(model_save_path, include_optimizer=False)

In [57]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

6572